# MC920 - Trabalho 1

Rafael Sartori M. Santos, 186154

## Problema

Aplicar técnicas de meios-tons com difusão de erros de forma a alterar os níveis de cinza das imagens dadas.

## Organização

* Trecho comum para lidar com imagem (funções auxiliares para salvar, mostrar imagens);
* Código base para o problema atual;
* Aplicações de filtros diferentes em todo conjunto de imagem;
* Isolamento de partes da imagem para destaques.

#### Observação:

Para executar corretamente todos os programas, é necessário executar o código inicial comum a todas as soluções.

## Soluções

Utilizarei `numpy` para executar as transformações de forma otimizada, interface OpenCV (`cv2` para Python) para abrir e salvar imagens no formato desejado e bibliotecas padrão para outras funções auxiliares.

In [1]:
# Lemos o código auxiliar (definições de funções auxiliares para abrir, salvar e mostrar imagens)
%run util/util.py

### Método geral

Para produzir meios-tons utilizando difusão de erro, fazemos:
* Abrimos imagem dada
* Conferimos se há filtro
    * Se não houver, aplicamos a binarização simples de forma vetorizada, salvamos e retornamos
    * Se houver, continuamos
* Adicionamos _padding_ à imagem de entrada
* Inicializamos uma imagem de saída de mesma dimensão que a original
* Percorremos a imagem de forma zigue-zague (da esq. para dir. nas linhas pares, da dir. para esq. nas linhas ímpares)
    * Fazemos binarização
    * Aplicamos o filtro de difusão de erro de forma vetorizada

Para realizar isso, criamos um _iterator_ que faz zigue-zague na imagem. Enquanto percorremos, definimos o nível de preto e branco na imagem final e somamos à imagem de entrada um filtro de difusão de erro multiplicado pelo erro na posição ajustada.

Por exemplo, para o filtro Floyd-Steinberg (representado em matriz retangular, onde asterisco é a posição de aplicação), temos a matriz:
```
filtro = [[0, '*', 1/4],
          [1/4, 1/4, 1/4]]
```
onde a posição de aplicação será o par ordenado da posição `(0, 1)` na matriz, que é substituído por `0` no filtro pois o ponto já foi calculado.

In [2]:
import time # para contar o tempo de execução de cada filtro

class ZigueZague:
    """Anda crescentemente em linhas pares e decrescentemente em linhas ímpares."""

    def __init__(self, formato, filtro, inicio=(0, 0)):
        self.linhas, self.colunas = formato
        # Verificamos o início na imagem
        y, x = inicio
        if (x >= self.colunas or x < 0 or y >= self.linhas or y < 0):
            raise ValueError('Posição inicial fora da imagem!')
        # Inicializamos o resto
        self.pos = inicio
        self.funcao_next = self.first_next
        self.filtro = filtro
        self.filtro_inv = np.flip(filtro)
    
    def first_next(self):
        y, x = self.pos
        
        # Conferimos se temos que iterar crescentemente
        if y % 2 == 0:
            self.funcao_next = self.crescente_next
            filtro = self.filtro
        else:
            self.funcao_next = self.decrescente_next
            filtro = self.filtro_inv
        
        # Retornamos a posição atual (sem atualizar, pois nunca havia sido mostrada)
        return (self.pos, filtro)
    
    def crescente_next(self):
        y, x = self.pos
        
        # Somamos uma posição (crescente)
        x += 1
        # Verificamos borda
        if x >= self.colunas:
            # Vamos à próxima linha
            y += 1
            
            # Conferimos se a imagem acabou
            if y >= self.linhas:
                raise StopIteration()
            
            # Voltamos dentro da imagem
            x = self.colunas - 1
            # Alteramos função para decrescente
            self.funcao_next = self.decrescente_next
        
        # Mostramos imagem atual
        self.pos = (y, x)
        return (self.pos, self.filtro)
    
    def decrescente_next(self):
        (y, x) = self.pos
        
        # Subtraímos uma posição (decrescente)
        x -= 1
        # Verificamos borda
        if x < 0:
            # Vamos à próxima linha
            y += 1
            
            # Conferimos se a imagem acabou
            if y >= self.linhas:
                raise StopIteration()
            
            # Voltamos dentro da imagem
            x = 0
            # Alteramos função para crescente
            self.funcao_next = self.crescente_next
        
        # Mostramos imagem atual
        self.pos = (y, x)
        return (self.pos, self.filtro_inv)

    def __iter__(self):
        return self
    
    def __next__(self):
        # Iteramos na função dada
        return self.funcao_next()
    
    def is_decrescendo(self):
        # Estamos decrescendo se a linha atual é ímpar
        return (self.pos[0] % 2) == 1
    
    def get_pos(self):
        return self.pos

    
def aplicar_halftoning(caminho_in, caminho_out, filtro=None):
    """Aplica halftoning na imagem colorida dada e salva em um caminho dado.
    O filtro de disseminação de erro deve ser a tupla (filtro, (x do centro, y do centro))
    onde centro é o local de aplicação"""
    
    # Carregamos a imagem de entrada
    entradas = abrir_imagem(caminho_in)
    # Salvamos o formato (consideramos que há pelo menos uma camada)
    img_linhas, img_colunas = img_formato = entradas[0].shape
    
    # Definimos a função para finalizar a imagem
    def finalizar(camadas):
        # Juntamos o canal em uma imagem
        saida = cv2.merge(camadas)

        # Salvamos a imagem
        salvar_imagem(caminho_out, saida)
    
    # Binarizamos a saída de forma simples se não há filtro
    if filtro is None:
        # Para cada camada (utilizamos entrada mesmo)
        for camada in entradas:
            # Os pixels maiores que 128 tornam-se 255
            camada[camada >= 128] = 255
            camada[camada < 128] = 0
        
        # Acabamos se não há filtro
        finalizar(entradas)
        return
    
    # Definimos padding padrão para o filtro
    pad_y_top, pad_y_bot = (0, 0)
    pad_x_left, pad_x_right = (0, 0)
    
    filtro_linhas, filtro_colunas = filtro_formato = (0, 0)
    centro_y, centro_x = filtro_centro = (0, 0)
    
    # Verificamos se há filtro e analisamos para determinar padding e processar
    if filtro is not None:
        # Pegamos as informações necessárias
        filtro, filtro_centro = filtro
        filtro = np.array(filtro)
        filtro_linhas, filtro_colunas = filtro_formato = filtro.shape
        centro_y, centro_x = filtro_centro
        
        # Atualizamos o padding com o formato do filtro
        pad_x_left, pad_x_right = (centro_x, filtro_colunas - centro_x - 1)
        pad_y_top, pad_y_bot = (centro_y, filtro_linhas - centro_y - 1)
        
    # Criamos o padding na entrada
    entradas = list(map(
        lambda camada: np.pad(camada, ((pad_y_top, pad_y_bot), (pad_x_left, pad_x_right)), constant_values=(0,)),
        entradas
    ))
    
    # Função para cada camada
    def aplicar_disseminacao(camada_in):
        # Fazemos nova matriz para saída
        camada_out = np.zeros(img_formato)
        
        # Iteramos em zigue-zague aplicando o filtro
        iterator = ZigueZague(img_formato, filtro)
        for (y, x), filtro_ in iterator:
            # Corrigimos y, x para entrada
            y_, x_ = (y + pad_y_top, x + pad_x_left)
            
            # Fazemos a binarização
            camada_out[y][x] = 255 if camada_in[y_][x_] >= 128 else 0
            
            # Calculamos o erro para propagar
            erro = camada_in[y_][x_] - camada_out[y][x]
            
            # Para cada posição, seccionamos a imagem do tamanho do filtro
            secao = camada_in[(y_ - pad_y_top):(y_ + pad_y_bot + 1), (x_ - pad_x_left):(x_ + pad_x_right + 1)]
            
            # Propagamos o erro
            secao += erro * filtro_
            # Limitamos o intervalo
            secao[secao > 255] = 255
            secao[secao < 0] = 0 # zeramos negativos
            #secao = np.absolute(secao) # refletimos para o positivo o que está negativo
        
        # Retornamos camada final
        return camada_out
    
    # Aplica difusão de erro na camada enquanto a binariza
    saidas = []
    for camada in entradas:
        saidas.append(aplicar_disseminacao(camada))
    
    # Salvamos a imagem final
    finalizar(saidas)

#### Filtro Floyd e Steinberg

Definimos o filtro:
```
filtro = [[0, '*', 7/16],
          [3/16, 5/16, 1/16]]
```
onde o asterisco é o centro e aplicamos às imagens.

In [3]:
# Definimos o filtro
filtro = ([[0, 0, 7/16],
           [3/16, 5/16, 1/16]], (0, 1))

start = time.process_time()
aplicar_halftoning('imgs/mel.png', 'imgs/mel_binarizada-floyd_steinberg.png', filtro)
end = time.process_time()
print('Tempo de execução:', end - start, 'segundos')

Tempo de execução: 833.96875 segundos


#### Filtro Stevenson e Arce

Definimos o filtro:
```
filtro = [[0, 0, 0, '*', 0, 32/200, 0],
          [12/200, 0, 26/200, 0, 30/200, 0, 16/200],
          [0, 12/200, 0, 26/200, 0, 12/200, 0],
          [5/200, 0, 12/200, 0, 12/200, 0, 5/200]]

```
onde o asterisco é o centro e aplicamos às imagens.

In [4]:
# Definimos o filtro
filtro = ([[0, 0, 0, 0, 0, 32/200, 0],
           [12/200, 0, 26/200, 0, 30/200, 0, 16/200],
           [0, 12/200, 0, 26/200, 0, 12/200, 0],
           [5/200, 0, 12/200, 0, 12/200, 0, 5/200]], (0, 3))

start = time.process_time()
aplicar_halftoning('imgs/mel.png', 'imgs/mel_binarizada-stevenson_arce.png', filtro)
end = time.process_time()
print('Tempo de execução:', end - start, 'segundos')

Tempo de execução: 906.6875 segundos


#### Filtro Burkes

Definimos o filtro:
```
filtro = [[0, 0, '*', 8/32, 4/32],
          [2/32, 4/32, 8/32, 4/32, 2/32]]
```
onde o asterisco é o centro e aplicamos às imagens.

In [5]:
# Definimos o filtro
filtro = ([[0, 0, 0, 8/32, 4/32],
           [2/32, 4/32, 8/32, 4/32, 2/32]], (0, 2))

start = time.process_time()
aplicar_halftoning('imgs/mel.png', 'imgs/mel_binarizada-burkes.png', filtro)
end = time.process_time()
print('Tempo de execução:', end - start, 'segundos')

Tempo de execução: 903.25 segundos


#### Filtro Sierra

Definimos o filtro:
```
filtro = [[0, 0, '*', 5/32, 3/32],
          [2/32, 4/32, 5/32, 4/32, 2/32],
          [0, 2/32, 3/32, 2/32, 0]]
```
onde o asterisco é o centro e aplicamos às imagens.

In [6]:
# Definimos o filtro
filtro = ([[0, 0, 0, 5/32, 3/32],
           [2/32, 4/32, 5/32, 4/32, 2/32],
           [0, 2/32, 3/32, 2/32, 0]], (0, 2))

start = time.process_time()
aplicar_halftoning('imgs/mel.png', 'imgs/mel_binarizada-sierra.png', filtro)
end = time.process_time()
print('Tempo de execução:', end - start, 'segundos')

Tempo de execução: 891.265625 segundos


#### Filtro Stucki

Definimos o filtro:
```
filtro = [[0, 0, '*', 8/42, 4/42],
          [2/42, 4/42, 8/42, 4/42, 2/42],
          [1/42, 2/42, 4/42, 2/42, 1/42]]
```
onde o asterisco é o centro e aplicamos às imagens.

In [7]:
# Definimos o filtro
filtro = ([[0, 0, 0, 8/42, 4/42],
           [2/42, 4/42, 8/42, 4/42, 2/42],
           [1/42, 2/42, 4/42, 2/42, 1/42]], (0, 2))

start = time.process_time()
aplicar_halftoning('imgs/mel.png', 'imgs/mel_binarizada-stucki.png', filtro)
end = time.process_time()
print('Tempo de execução:', end - start, 'segundos')

Tempo de execução: 918.84375 segundos


#### Filtro Jarvis, Judice e Ninke

Definimos o filtro:
```
filtro = [[0, 0, '*', 8/42, 4/42],
          [2/42, 4/42, 8/42, 4/42, 2/42],
          [1/42, 2/42, 4/42, 2/42, 1/42]]
```
onde o asterisco é o centro e aplicamos às imagens.

In [8]:
# Definimos o filtro, começando no próximo item em relação
# ao próximo do que foi aplicado
filtro = ([[0, 0, 0, 7/48, 5/48],
           [3/48, 5/48, 7/48, 5/48, 3/48],
           [1/48, 3/48, 5/48, 3/48, 1/48]], (0, 2))

start = time.process_time()
aplicar_halftoning('imgs/mel.png', 'imgs/mel_binarizada-jarvis_judice_ninke.png', filtro)
end = time.process_time()
print('Tempo de execução:', end - start, 'segundos')

Tempo de execução: 908.359375 segundos


#### Selecionamos trechos de destaque

Para produzirmos comparações interessantes no relatório, precisamos selecionar partes pequenas e na mesma posição entre várias imagens. Abriremos, então, várias imagens e salvamos um recorte em posição separada.

In [9]:
import glob # para abrir vários arquivos


# Cortamos a imagem na posição (canto diagonal superior) + tamanho e salvamos
# Posição, tamanho possuem formato (y, x)
def crop(caminho_in, posicao, tamanho, caminho_out):
    # Carregamos a imagem de entrada
    entradas = abrir_imagem(arquivo)
    
    # Pegamos as coordenadas e tamanho
    pos_y, pos_x = posicao
    tam_y, tam_x = tamanho
    
    # Para cada camada...
    saidas = []
    for camada in entradas:
        saidas.append(camada[pos_y:(pos_y + tam_y), pos_x:(pos_x + tam_x)])
    
    # Juntamos o canal em uma imagem
    saida = cv2.merge(saidas)
    # Salvamos a imagem final
    salvar_imagem(caminho_out, saida)
    

# Pegamos todas as imagens PNG
for arquivo in glob.glob('imgs/mel*.png'):
    # Ignoramos arquivos que são detalhes (para não fazermos recursivamente)
    if "detalhe" in arquivo:
        continue
    
    # Fazemos diversos crops
    for i, (posicao, tamanho) in enumerate([((1023, 2037), (492, 492)), ((393, 1122), (372, 372))]):
        # Fazemos o crop e salvamos
        arquivo_final = arquivo.replace('.png', '-detalhe{0}.png'.format(i + 1))
        crop(arquivo, posicao, tamanho, arquivo_final)

        # Anunciamos
        print(arquivo, '@', posicao, '+', tamanho, '->', arquivo_final)

imgs/mel-binarizada.png @ (1023, 2037) + (492, 492) -> imgs/mel-binarizada-detalhe1.png
imgs/mel-binarizada.png @ (393, 1122) + (372, 372) -> imgs/mel-binarizada-detalhe2.png
imgs/mel.png @ (1023, 2037) + (492, 492) -> imgs/mel-detalhe1.png
imgs/mel.png @ (393, 1122) + (372, 372) -> imgs/mel-detalhe2.png
imgs/mel_binarizada-burkes.png @ (1023, 2037) + (492, 492) -> imgs/mel_binarizada-burkes-detalhe1.png
imgs/mel_binarizada-burkes.png @ (393, 1122) + (372, 372) -> imgs/mel_binarizada-burkes-detalhe2.png
imgs/mel_binarizada-floyd_steinberg.png @ (1023, 2037) + (492, 492) -> imgs/mel_binarizada-floyd_steinberg-detalhe1.png
imgs/mel_binarizada-floyd_steinberg.png @ (393, 1122) + (372, 372) -> imgs/mel_binarizada-floyd_steinberg-detalhe2.png
imgs/mel_binarizada-jarvis_judice_ninke.png @ (1023, 2037) + (492, 492) -> imgs/mel_binarizada-jarvis_judice_ninke-detalhe1.png
imgs/mel_binarizada-jarvis_judice_ninke.png @ (393, 1122) + (372, 372) -> imgs/mel_binarizada-jarvis_judice_ninke-detalhe2.p